# Experiment Documentation: Assessing Positional Congruence in Keypoint Detection

This Colab notebook evaluates the **repeatability** of keypoint detection algorithms by measuring **positional congruence** between detected keypoints in original images (`I`) and their transformed counterparts (`τ(I)`). The experiment compares our method's performance against state-of-the-art detectors, including **KeyNet** and **REKD**.

### Methodology:
The evaluation is restricted to overlapping subregions between the original and transformed datasets to ensure meaningful comparisons. The criterion for positional congruence is defined as:

**|Kᵢ(τ(Iᵢ)) - τ(Kᵢ(Iᵢ))| ≤ α**

Where:
- `Kᵢ(Iᵢ)`: Keypoints detected in the original image.
- `Kᵢ(τ(Iᵢ))`: Keypoints detected in the transformed image.
- `τ`: Transformation applied to the image.
- `α`: Acceptable positional deviation threshold.

### Objective:
This experiment aims to validate the hypothesis that improving keypoint repeatability enhances feature matching and image identification accuracy. It forms the foundation for comparing detector performance and evaluating their robustness to image transformations.

By adopting a rigorous and reproducible framework, this notebook provides an impartial assessment of keypoint detectors under diverse real-world conditions.


In [3]:
import torch
import kornia
from torch import nn
from teste_util import *
# from kornia.feature.keynet import KeyNetDetector
# from custom_local_feature import REKDSosNet, SingularPointSosNet
from LocalFeatureCombinations import KeyNetFeatureSIFT, REKDSosNet, SingularPointSosNet
import itertools

# Fixar a semente do Torch para operações específicas
set_seed(42)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


keynet_default_config = {
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.0, 's_mult': 5.0},
}


# Inicializar os detectores e adicionar seus nomes de classe
detectors = {
    # "KeyNetDetector": KeyNetFeatureSIFT(config=keynet_default_config,device=device).initialize_detector(num_features=60).to(device),
    "REKDSosNet": REKDSosNet(config=keynet_default_config,device=device).initialize_detector(num_features=60).to(device),
    # "SingularPointSosNet": SingularPointSosNet(config=keynet_default_config,device=device).initialize_detector(num_features=60).to(device),
}

# Leitura dos dados
dataloaders = {
    # "flower": read_dataload_flower(120)[1],
    # "fibers": read_dataload_fibers(120)[1],
    "woods": read_dataload_woods(120)[1],
}

# Gerar combinações entre detectores e dataloaders
def generate_combinations(detectors, dataloaders):
    # Produto cartesiano entre detectores e dataloaders
    return list(itertools.product(dataloaders.items(), detectors.items()))

# Gerar combinações
combinations = generate_combinations(detectors, dataloaders)

# Listar as combinações
# for i, ((dataset_name, dataloader), (class_name, detector)) in enumerate(combinations, 1):
#     print(f"Combinação {i}: Detector - {class_name} ({detector.__class__.__name__}), Dataset - {dataset_name}")


train True   598
train False   7946
7920


In [4]:
import numpy as np
from scipy.spatial.distance import cdist
import kornia.feature as KF
import matplotlib.pyplot as plt
import kornia as K

def visualize_LAF(img, LAF, img_idx = 0):
    x, y = KF.laf.get_laf_pts_to_draw(LAF, img_idx)
    print(x[0][:5],y[0][:5])
    plt.figure()
    plt.imshow(K.utils.tensor_to_image(img[img_idx]))
    plt.plot(x, y, 'r')
    plt.show()
    return

import matplotlib.pyplot as plt
import numpy as np

def visualize_matching_images(img1, LAF1, img2, LAF2, matches):
    print(LAF1.shape,LAF2.shape)
    x1, y1 = KF.laf.get_laf_pts_to_draw(LAF1, 0)  # Pontos da primeira imagem
    x2, y2 = KF.laf.get_laf_pts_to_draw(LAF2, 0)  # Pontos da segunda imagem

    # Converte as listas x2 e y2 em arrays NumPy
    x2 = np.array(x2)
    y2 = np.array(y2)

    # Crie uma imagem combinada concatenando as duas imagens lado a lado
    combined_image = np.concatenate((K.utils.tensor_to_image(img1), K.utils.tensor_to_image(img2)), axis=1)

    fig, ax = plt.subplots(1, 1, figsize=(12, 6))  # Cria uma figura com uma subplot

    # Plota a imagem combinada
    ax.imshow(combined_image)
    ax.axis('off')
    # Plota os pontos correspondentes nas duas imagens
    ax.plot(x1, y1, 'c')  # 'ro' representa pontos vermelhos na primeira imagem
    ax.plot(x2 + img1.shape[1], y2, 'y')  # Desloca os pontos azuis na segunda imagem para a direita

    points1 =kornia.feature.get_laf_center(LAF1)[0].cpu()
    points2 =kornia.feature.get_laf_center(LAF2)[0].cpu()
    print(points1.shape,points2.shape)
    for match in matches:
        x1_match, y1_match = points1[match[0],0], points1[match[0],1]
        x2_match, y2_match = points2[match[1],0] + img1.shape[1], points2[match[1],1]
       
        ax.plot([x1_match, x2_match], [y1_match, y2_match], '-', color='red', lw=1.5)

    plt.tight_layout()
    plt.show()

    
def plot_matches_keypoints(image1, keypoints1, image2, keypoints2, matches, **kwargs):
    print('image1 shape: ',image1.shape,image1.dtype,image2.shape,image2.dtype)
    # Concatenar as duas imagens lado a lado
    combined_image = np.concatenate((image1, image2), axis=1)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(combined_image)
    ax.axis('off')

    # Desenhar pontos correspondentes e linhas conectando-os
    offset = image1.shape[1]

    for i, (x, y) in enumerate(keypoints1):
        ax.plot(x, y, 'o',markerfacecolor='none', markeredgecolor='r',
                markersize=20, markeredgewidth=1)
        ax.annotate(str(i), (x, y), color='r',xytext=(10, 10), textcoords='offset points', fontsize=12)

    for i, (x, y) in enumerate(keypoints2):
        ax.plot(x+offset, y, 'o',markerfacecolor='none', markeredgecolor='r',
                markersize=20, markeredgewidth=1)
        ax.annotate(str(i), (x+offset, y), color='r',xytext=(10, 10), textcoords='offset points', fontsize=12)

    for match in matches:
        x1, y1 = keypoints1[match[0],0], keypoints1[match[0],1]
        x2, y2 = keypoints2[match[1],0]+offset, keypoints2[match[1],1]
        ax.plot([x1, x2], [y1, y2], '-', color='lime', lw=0.5)

    plt.tight_layout()
    plt.show()

def plot_image_with_keypoints(image_tensor, keypoints_tensor):
    # Converter a imagem tensorial em objeto PIL.Image
    image = kornia.utils.tensor_to_image(image_tensor)
    # Plotar a imagem e os keypoints
    plt.imshow(image)
    if keypoints_tensor is not None:
        # Extrair as coordenadas x e y dos keypoints
        keypoints_x = keypoints_tensor[:,0].flatten().tolist()
        keypoints_y = keypoints_tensor[:,1].flatten().tolist()
        plt.scatter(keypoints_x, keypoints_y, c='red')
    plt.show()
    
def filtrar_keypoints(lista_de_pontos, tensor_mascara):
    # Verificar se as coordenadas estão dentro das dimensões
    dimensao_max_x, dimensao_max_y = tensor_mascara.shape[1] - 1, tensor_mascara.shape[0] - 1
    pontos_filtrados = [
        ponto.tolist()  for ponto in lista_de_pontos 
        if 0 <= ponto[0] <= dimensao_max_x 
        and 0 <= ponto[1] <= dimensao_max_y 
        and tensor_mascara[int(ponto[1]), int(ponto[0])] 
    ]
    return torch.tensor(pontos_filtrados)

def find_best_matching_indices_knn(points1, points2, threshold, k=3):
    if len(points1) == 0 or len(points2) == 0:
        return []
    distances = cdist(points1, points2)
    best_indices = np.argsort(distances, axis=1)[:, :k]
    best_distances = np.take_along_axis(distances, best_indices, axis=1)
    matched = []

    for i in range(len(points1)):
        min_distance = np.min(best_distances[i])        
        if min_distance < threshold:
            best_index = np.argmin(best_distances[i])
            matched.append((i, best_indices[i, best_index]))

    return matched


def detect_extract_feat_in_batch(batch1,aug_list, detector):
    origem_total = []
    matches_total = []
    with torch.no_grad():
        for img1  in batch1:            
            lafs1, resps1 = detector(img1[None])

            B,C,H,W = img1[None].shape
            mask = torch.ones(B,C,H,W).to(img1.device) 
            
            #lafs1 to points1
            points1 =kornia.feature.get_laf_center(lafs1)

            if( points1.shape[1] == 0):
                print('aug_list shape: ',points1.shape) 
                continue     

            params = next(aug_list)    
            img2,mask_t,ponts_t=aug_list.augmentation_sequence(img1,mask,points1,params=params)           
             
            img2 = img2.to(img1.device)
            lafs2, resps2 = detector(img2)
            points2 =kornia.feature.get_laf_center(lafs2)     
            filtered_points1 = filtrar_keypoints(ponts_t[0],mask_t[0,0].bool())            
            filtered_points2 = filtrar_keypoints(points2[0],mask_t[0,0].bool())
            # print("shape p & f",points1.shape,filtered_points1.shape,filtered_points2.shape)
            matches = find_best_matching_indices_knn(filtered_points1.cpu(), filtered_points2.cpu(), threshold=1.0, k=1)  

            if( filtered_points1.shape[0] == 0 or filtered_points2.shape[0] == 0):
                print('filtered_points1 shape: ',filtered_points1.shape,'filtered_points2 shape: ',filtered_points2.shape)
                continue       
                 
            origem_total.append(len(filtered_points1))
            matches_total.append(len(matches))            
    return (np.mean(matches_total)/np.mean(origem_total))*100

In [5]:
class AugmentationGenerator:
    def __init__(self, n_variations):
        # Definir as augmentações
        aug_gen = kornia.augmentation.AugmentationSequential(
            kornia.augmentation.RandomAffine(degrees=360, translate=(0.2, 0.2), scale=(0.95, 1.05), shear=10,p=0.8),
            kornia.augmentation.RandomPerspective(0.2, p=0.7),
            kornia.augmentation.RandomBoxBlur((4,4),p=0.5),
            data_keys=[kornia.constants.DataKey.INPUT,  # Especifica as chaves corretamente
                       kornia.constants.DataKey.MASK,
                       kornia.constants.DataKey.KEYPOINTS],
            same_on_batch=True,
        )

        self.augmentation_sequence = aug_gen
        self.n_variations = n_variations
        self.param_list = []
        self.current_index = 0

    def generate_variations(self, image, mask, keypoints):
        """
        Gera múltiplas variações de augmentações e coleta seus parâmetros.
        """
        for _ in range(self.n_variations):
            # Apenas executa a sequência de augmentação e salva os parâmetros gerados
            self.augmentation_sequence(image, mask, keypoints)
            self.param_list.append(self.augmentation_sequence._params)

    def __iter__(self):
        self.current_index = 0  # Resetar o índice a cada nova iteração
        return self

    def __next__(self):
        """
        Retorna a próxima variação de parâmetros de augmentação.
        A iteração será circular.
        """
        result = self.param_list[self.current_index]
        self.current_index = (self.current_index + 1) % len(self.param_list)
        return result

    def reset(self):
        """Método para resetar o estado do gerador de augmentação."""
        self.current_index = 0  # Reseta o índice de iteração

In [6]:
from tqdm.notebook import tqdm


set_seed(42)

aug_gen = AugmentationGenerator(15)
image = torch.rand(1, 1, 120, 120)  # Imagem com valores aleatórios
mask = torch.ones(1, 1, 120, 120)  # Máscara binária
keypoints = torch.tensor([[[30, 30], [90, 90]]], dtype=torch.float32)  # Pontos chave de exemplo

# Gerar as variações
aug_gen.generate_variations(image, mask, keypoints)

for i, ((dataset_name, dataloader), (class_name, detector)) in enumerate(combinations, 1):
    matches_total = []
    pbar = tqdm(dataloader, desc=f"Evaluation {class_name}-{dataset_name}")  # Usando f-string para formatar o nome da classe do detector
    for imgs_batch, labels_batch in pbar:  # Itera em todo o dataset
        imgs_batch = imgs_batch.to(device)    
        mean = detect_extract_feat_in_batch(imgs_batch, aug_gen, detector)
        matches_total.append(mean)
        pbar.set_postfix({"Dataset Match Mean": f"{np.mean(matches_total):.4f}"})
        aug_gen.reset()


/home/wagner/miniconda3/envs/singular-points/lib/python3.9/site-packages/torch/nn/functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/wagner/miniconda3/envs/singular-points/lib/python3.9/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Evaluation REKDSosNet-woods:   0%|          | 0/256 [00:00<?, ?it/s]

filtered_points1 shape:  torch.Size([33, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([48, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([13, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([31, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([14, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([15, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([37, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([32, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([27, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([46, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([18, 2]) filtered_points2 shape:  torch.Size([0])
filtered_points1 shape:  torch.Size([17, 2]) filtered_

Evaluation KeyNetDetector-flower: 100% 102/102 [05:33<00:00,  2.52s/it, Dataset Match Mean=11.0770]
Evaluation REKDSosNet-flower: 100% 102/102 [05:05<00:00,  2.32s/it, Dataset Match Mean=30.2933]
Evaluation SingularPointSosNet-flower: 100% 102/102 [07:20<00:00,  3.29s/it, Dataset Match Mean=40.1591]

Evaluation KeyNetDetector-fibers: 100% 9/9 [00:18<00:00,  2.03s/it, Dataset Match Mean=10.4233]
Evaluation REKDSosNet-fibers: 100% 9/9 [00:14<00:00,  1.57s/it, Dataset Match Mean=3.6245]
Evaluation SingularPointSosNet-fibers: 100% 9/9 [00:29<00:00,  3.26s/it, Dataset Match Mean=42.1785]

Evaluation KeyNetDetector-woods: 100% 256/256 [07:16<00:00,  1.44s/it, Dataset Match Mean=13.1803]
Evaluation REKDSosNet-woods: 100% 256/256 [06:11<00:00,  1.17s/it, Dataset Match Mean=21.7026]
Evaluation SingularPointSosNet-woods: 100% 256/256 [09:06<00:00,  1.92s/it, Dataset Match Mean=37.6250]